# Mini-batch preprocessor

The mini-batch preprocessor is a utility that prepares input data for use by models that support mini-batch as an optimization option. (This is currently only the case for Neural Networks.) It is effectively a packing operation that builds arrays of dependent and independent variables from the source data table.

The mini-batch preprocessor was added in MADlib 1.14.

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# Greenplum Database 5.4.0 on GCP (demo machine)
%sql postgresql://gpadmin@35.184.253.255:5432/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

# Greenplum Database 4.3.10.0
#%sql postgresql://gpdbchina@10.194.10.68:61000/madlib

u'Connected: gpadmin@madlib'

In [4]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.14-dev, git revision: rc/1.13-rc1-66-g4cced1b, cmake configuration time: Mon Apr 23 16:26:17 UTC 2018, build type: release, build system: Linux-2.6.32-696.20.1.el6.x86_64, C compiler: gcc 4.4.7, C++ compiler: g++ 4.4.7"


# 1. Load data
Based on the well known iris dataset.

In [6]:
%%sql
DROP TABLE IF EXISTS iris_data;

CREATE TABLE iris_data(
    id serial,
    attributes numeric[],
    class_text varchar,
    class integer,
    state varchar
);

INSERT INTO iris_data(id, attributes, class_text, class, state) VALUES
(1,ARRAY[5.0,3.2,1.2,0.2],'Iris_setosa',1,'Alaska'),
(2,ARRAY[5.5,3.5,1.3,0.2],'Iris_setosa',1,'Alaska'),
(3,ARRAY[4.9,3.1,1.5,0.1],'Iris_setosa',1,'Alaska'),
(4,ARRAY[4.4,3.0,1.3,0.2],'Iris_setosa',1,'Alaska'),
(5,ARRAY[5.1,3.4,1.5,0.2],'Iris_setosa',1,'Alaska'),
(6,ARRAY[5.0,3.5,1.3,0.3],'Iris_setosa',1,'Alaska'),
(7,ARRAY[4.5,2.3,1.3,0.3],'Iris_setosa',1,'Alaska'),
(8,ARRAY[4.4,3.2,1.3,0.2],'Iris_setosa',1,'Alaska'),
(9,ARRAY[5.0,3.5,1.6,0.6],'Iris_setosa',1,'Alaska'),
(10,ARRAY[5.1,3.8,1.9,0.4],'Iris_setosa',1,'Alaska'),
(11,ARRAY[4.8,3.0,1.4,0.3],'Iris_setosa',1,'Alaska'),
(12,ARRAY[5.1,3.8,1.6,0.2],'Iris_setosa',1,'Alaska'),
(13,ARRAY[5.7,2.8,4.5,1.3],'Iris_versicolor',2,'Alaska'),
(14,ARRAY[6.3,3.3,4.7,1.6],'Iris_versicolor',2,'Alaska'),
(15,ARRAY[4.9,2.4,3.3,1.0],'Iris_versicolor',2,'Alaska'),
(16,ARRAY[6.6,2.9,4.6,1.3],'Iris_versicolor',2,'Alaska'),
(17,ARRAY[5.2,2.7,3.9,1.4],'Iris_versicolor',2,'Alaska'),
(18,ARRAY[5.0,2.0,3.5,1.0],'Iris_versicolor',2,'Alaska'),
(19,ARRAY[5.9,3.0,4.2,1.5],'Iris_versicolor',2,'Alaska'),
(20,ARRAY[6.0,2.2,4.0,1.0],'Iris_versicolor',2,'Alaska'),
(21,ARRAY[6.1,2.9,4.7,1.4],'Iris_versicolor',2,'Alaska'),
(22,ARRAY[5.6,2.9,3.6,1.3],'Iris_versicolor',2,'Alaska'),
(23,ARRAY[6.7,3.1,4.4,1.4],'Iris_versicolor',2,'Alaska'),
(24,ARRAY[5.6,3.0,4.5,1.5],'Iris_versicolor',2,'Alaska'),
(25,ARRAY[5.8,2.7,4.1,1.0],'Iris_versicolor',2,'Alaska'),
(26,ARRAY[6.2,2.2,4.5,1.5],'Iris_versicolor',2,'Alaska'),
(27,ARRAY[5.6,2.5,3.9,1.1],'Iris_versicolor',2,'Alaska'),
(28,ARRAY[5.0,3.4,1.5,0.2],'Iris_setosa',1,'Tennessee'),
(29,ARRAY[4.4,2.9,1.4,0.2],'Iris_setosa',1,'Tennessee'),
(30,ARRAY[4.9,3.1,1.5,0.1],'Iris_setosa',1,'Tennessee'),
(31,ARRAY[5.4,3.7,1.5,0.2],'Iris_setosa',1,'Tennessee'),
(32,ARRAY[4.8,3.4,1.6,0.2],'Iris_setosa',1,'Tennessee'),
(33,ARRAY[4.8,3.0,1.4,0.1],'Iris_setosa',1,'Tennessee'),
(34,ARRAY[4.3,3.0,1.1,0.1],'Iris_setosa',1,'Tennessee'),
(35,ARRAY[5.8,4.0,1.2,0.2],'Iris_setosa',1,'Tennessee'),
(36,ARRAY[5.7,4.4,1.5,0.4],'Iris_setosa',1,'Tennessee'),
(37,ARRAY[5.4,3.9,1.3,0.4],'Iris_setosa',1,'Tennessee'),
(38,ARRAY[6.0,2.9,4.5,1.5],'Iris_versicolor',2,'Tennessee'),
(39,ARRAY[5.7,2.6,3.5,1.0],'Iris_versicolor',2,'Tennessee'),
(40,ARRAY[5.5,2.4,3.8,1.1],'Iris_versicolor',2,'Tennessee'),
(41,ARRAY[5.5,2.4,3.7,1.0],'Iris_versicolor',2,'Tennessee'),
(42,ARRAY[5.8,2.7,3.9,1.2],'Iris_versicolor',2,'Tennessee'),
(43,ARRAY[6.0,2.7,5.1,1.6],'Iris_versicolor',2,'Tennessee'),
(44,ARRAY[5.4,3.0,4.5,1.5],'Iris_versicolor',2,'Tennessee'),
(45,ARRAY[6.0,3.4,4.5,1.6],'Iris_versicolor',2,'Tennessee'),
(46,ARRAY[6.7,3.1,4.7,1.5],'Iris_versicolor',2,'Tennessee'),
(47,ARRAY[6.3,2.3,4.4,1.3],'Iris_versicolor',2,'Tennessee'),
(48,ARRAY[5.6,3.0,4.1,1.3],'Iris_versicolor',2,'Tennessee'),
(49,ARRAY[5.5,2.5,4.0,1.3],'Iris_versicolor',2,'Tennessee'),
(50,ARRAY[5.5,2.6,4.4,1.2],'Iris_versicolor',2,'Tennessee'),
(51,ARRAY[6.1,3.0,4.6,1.4],'Iris_versicolor',2,'Tennessee'),
(52,ARRAY[5.8,2.6,4.0,1.2],'Iris_versicolor',2,'Tennessee');

SELECT * FROM iris_data ORDER BY id;

Done.
Done.
52 rows affected.
52 rows affected.


id,attributes,class_text,class,state
1,"[Decimal('5.0'), Decimal('3.2'), Decimal('1.2'), Decimal('0.2')]",Iris_setosa,1,Alaska
2,"[Decimal('5.5'), Decimal('3.5'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
3,"[Decimal('4.9'), Decimal('3.1'), Decimal('1.5'), Decimal('0.1')]",Iris_setosa,1,Alaska
4,"[Decimal('4.4'), Decimal('3.0'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
5,"[Decimal('5.1'), Decimal('3.4'), Decimal('1.5'), Decimal('0.2')]",Iris_setosa,1,Alaska
6,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1,Alaska
7,"[Decimal('4.5'), Decimal('2.3'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1,Alaska
8,"[Decimal('4.4'), Decimal('3.2'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
9,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.6'), Decimal('0.6')]",Iris_setosa,1,Alaska
10,"[Decimal('5.1'), Decimal('3.8'), Decimal('1.9'), Decimal('0.4')]",Iris_setosa,1,Alaska


# 2.  Run preprocessor 

Run the preprocessor to generate the packed output table:

In [5]:
%%sql
DROP TABLE IF EXISTS iris_data_packed, iris_data_packed_summary, iris_data_packed_standardization;

SELECT madlib.minibatch_preprocessor('iris_data',         -- Source table
                                     'iris_data_packed',  -- Output table
                                     'class_text',        -- Dependent variable
                                     'attributes'        -- Independent variables
                                     );

SELECT * FROM iris_data_packed ORDER BY __id__;

Done.
1 rows affected.
2 rows affected.


__id__,dependent_varname,independent_varname
0,"[[1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0]]","[[-1.10156217454914, 0.00385956572525086, -1.14565239753098, -1.00286528298202], [-0.767560815504508, 0.806649237861967, -1.07515071152907, -1.18456909732025], [-0.0995580974152422, 0.00385956572525086, 1.03989986852812, 1.17758048907675], [-0.433559456459875, -0.598232688377286, 0.616889752516682, 0.995876674738521], [-0.934561495026824, 0.20455698375943, -1.07515071152907, -1.36627291165848], [1.23644733876329, -1.60171977854818, 1.03989986852812, 1.17758048907675], [1.06944665924097, -0.196837852308928, 1.18090324053193, 0.995876674738521], [0.0674425821070736, -0.798930106411465, 0.969398182526215, 0.632469046062059], [0.568444620674023, -0.598232688377286, 0.616889752516682, 0.632469046062059], [-0.600560135982193, 1.60943890999868, -0.793143967521448, -0.821161468643789], [-1.60256421311609, -1.401022360514, -1.21615408353289, -1.00286528298202], [-0.600560135982193, 0.806649237861967, -1.07515071152907, -1.18456909732025], [-0.0995580974152422, 1.81013632803286, -1.21615408353289, -0.821161468643789], [0.401443941151707, -0.397535270343108, 1.03989986852812, 0.81417286040029], [-0.767560815504508, -2.00311461461654, 0.334883008509056, 0.269061417385597], [0.234443261629389, -0.196837852308928, 0.405384694510963, 0.81417286040029], [1.06944665924097, 0.00385956572525086, 1.11040155453003, 0.995876674738521], [1.4034480182856, -1.401022360514, 0.969398182526215, 0.81417286040029], [-1.93656557216072, 0.00385956572525086, -1.3571574555367, -1.36627291165848], [0.0674425821070736, -1.20032494247982, 0.546388066514775, 0.450765231723828], [0.0674425821070736, -1.20032494247982, 0.475886380512869, 0.269061417385597], [2.07145073637487, 0.20455698375943, 0.969398182526215, 0.995876674738521], [0.73544530019634, 0.00385956572525086, 0.828394810522402, 1.17758048907675], [1.4034480182856, 0.605951819827788, 1.18090324053193, 1.35928430341498], [0.902445979718656, -0.196837852308928, 1.03989986852812, 1.17758048907675], [-0.934561495026824, -1.20032494247982, 0.193879636505243, 0.269061417385597]]"
1,"[[1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0]]","[[-0.767560815504508, 1.00734665589615, -1.21615408353289, -1.00286528298202], [-0.934561495026824, 0.20455698375943, -1.07515071152907, -1.36627291165848], [0.568444620674023, -0.798930106411465, 0.687391438518589, 0.632469046062059], [0.568444620674023, -0.598232688377286, 0.757893124520495, 0.269061417385597], [0.401443941151707, 2.81362341820376, -1.07515071152907, -0.821161468643789], [0.902445979718656, -1.60171977854818, 0.687391438518589, 0.269061417385597], [0.234443261629389, 0.00385956572525086, 0.757893124520495, 0.81417286040029], [-1.10156217454914, 0.806649237861967, -1.00464902552717, -1.18456909732025], [-1.76956489263841, 0.00385956572525086, -1.21615408353289, -1.18456909732025], [0.234443261629389, -0.999627524445644, 0.616889752516682, 0.450765231723828], [-0.767560815504508, 0.405254401793609, -1.28665576953479, -1.18456909732025], [-0.600560135982193, 1.60943890999868, -1.00464902552717, -1.18456909732025], [1.90445005685255, -0.196837852308928, 1.11040155453003, 0.81417286040029], [-0.767560815504508, 1.00734665589615, -1.00464902552717, -0.457753839967327], [0.234443261629389, 0.00385956572525086, 1.03989986852812, 1.17758048907675], [0.0674425821070736, 1.00734665589615, -1.21615408353289, -1.18456909732025], [-1.76956489263841, 0.40525440179

For small datasets like in this example, buffer size is mainly determined by the number of segments in the database. For a Greenplum database with 2 segments, there will be 2 rows with a buffer size of 26. For PostgresSQL, there would be only one row with a buffer size of 52 since it is a single node database. For larger data sets, other factors go into computing buffers size besides number of segments. 

Also, note above that the dependent variable has been one-hot encoded since it is categorical. Here is a sample of the packed output table

Review the output summary table:

In [6]:
%%sql
SELECT * FROM iris_data_packed_summary;

1 rows affected.


source_table,output_table,dependent_varname,independent_varname,dependent_vartype,buffer_size,class_values,num_rows_processed,num_missing_rows_skipped,grouping_cols
iris_data,iris_data_packed,class_text,attributes,character varying,26,"[u'Iris_setosa', u'Iris_versicolor']",52,0,None


Review the output standardization table:

In [7]:
%%sql
SELECT * FROM iris_data_packed_standardization;

1 rows affected.


mean,std
"[5.45961538462, 2.99807692308, 3.025, 0.851923076923]","[0.598799958695, 0.498262513686, 1.41840579525, 0.550346179381]"


# 3.  Change buffer size 

Generally the default buffer size will work well, but if you have occasion to change it:

In [8]:
%%sql
DROP TABLE IF EXISTS iris_data_packed, iris_data_packed_summary, iris_data_packed_standardization;

SELECT madlib.minibatch_preprocessor('iris_data',         -- Source table
                                     'iris_data_packed',  -- Output table
                                     'class_text',        -- Dependent variable
                                     'attributes',        -- Independent variables
                                     NULL,                -- Grouping
                                     10                   -- Buffer size
                                     );

SELECT * FROM iris_data_packed ORDER BY __id__;

Done.
1 rows affected.
6 rows affected.


__id__,dependent_varname,independent_varname
0,"[[0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0]]","[[-0.0995580974152422, 0.00385956572525086, 1.03989986852812, 1.17758048907675], [-0.767560815504508, 1.00734665589615, -1.00464902552717, -0.457753839967327], [-0.934561495026824, 0.20455698375943, -1.07515071152907, -1.36627291165848], [0.568444620674023, -0.798930106411465, 0.687391438518589, 0.632469046062059], [-0.767560815504508, 0.405254401793609, -1.28665576953479, -1.18456909732025], [-0.767560815504508, -2.00311461461654, 0.334883008509056, 0.269061417385597], [2.07145073637487, 0.20455698375943, 0.969398182526215, 0.995876674738521], [0.401443941151707, 2.81362341820376, -1.07515071152907, -0.821161468643789], [-0.934561495026824, 0.20455698375943, -1.07515071152907, -1.36627291165848], [0.0674425821070736, -1.20032494247982, 0.475886380512869, 0.269061417385597]]"
1,"[[0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0]]","[[0.0674425821070736, -0.999627524445644, 0.687391438518589, 0.81417286040029], [-0.0995580974152422, 1.4087414919645, -1.07515071152907, -1.18456909732025], [-0.0995580974152422, 1.81013632803286, -1.21615408353289, -0.821161468643789], [1.06944665924097, 0.00385956572525086, 1.11040155453003, 0.995876674738521], [0.0674425821070736, -0.798930106411465, 0.969398182526215, 0.632469046062059], [-1.10156217454914, 0.806649237861967, -1.00464902552717, -1.18456909732025], [-1.10156217454914, 0.00385956572525086, -1.14565239753098, -1.00286528298202], [-0.600560135982193, 1.60943890999868, -1.00464902552717, -1.18456909732025], [0.902445979718656, -0.598232688377286, 1.46290998453956, 1.35928430341498], [0.401443941151707, -0.798930106411465, 0.334883008509056, 0.269061417385597]]"
2,"[[1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0]]","[[-1.10156217454914, 0.00385956572525086, -1.14565239753098, -1.36627291165848], [0.234443261629389, 0.00385956572525086, 1.03989986852812, 1.17758048907675], [0.902445979718656, -0.196837852308928, 1.03989986852812, 1.17758048907675], [2.07145073637487, 0.20455698375943, 1.18090324053193, 1.17758048907675], [1.90445005685255, -0.196837852308928, 1.11040155453003, 0.81417286040029], [-0.600560135982193, 0.806649237861967, -1.07515071152907, -1.18456909732025], [-0.433559456459875, -0.598232688377286, 0.616889752516682, 0.995876674738521], [0.73544530019634, 0.00385956572525086, 0.828394810522402, 1.17758048907675], [1.06944665924097, -0.196837852308928, 1.18090324053193, 0.995876674738521], [-0.767560815504508, 1.00734665589615, -1.21615408353289, -1.00286528298202]]"
3,"[[0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0]]","[[0.902445979718656, 0.806649237861967, 1.03989986852812, 1.35928430341498], [1.4034480182856, -1.401022360514, 0.969398182526215, 0.81417286040029], [0.568444620674023, -0.598232688377286, 0.616889752516682, 0.632469046062059], [1.4034480182856, 0.605951819827788, 1.18090324053193, 1.35928430341498], [-1.60256421311609, -1.401022360514, -1.21615408353289, -1.00286528298202], [-1.76956489263841, -0.196837852308928, -1.14565239753098, -1.18456909732025], [0.0674425821070736, -1.20032494247982, 0.546388066514775, 0.450765231723828], [0.0674425821070736, 1.00734665589615, -1.21615408353289, -1.18456909732025], [-1.76956489263841, 0.405254401793609, -1.21615408353289, -1.18456909732025], [0.234443261629389, -0.999627524445644, 0.616889752516682, 0.450765231723828]]"
4,"[[1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0]]","[[0.568444620674023, 2.01083374606704, -1.28665576953479, -1.18456909732025], [0.234443261629389, -0.196837852308928, 0.405384694510963, 0.81417286040029], [-0.934561495026824, -1.20032494247982, 0

Review the output summary data:

In [9]:
%%sql
SELECT * FROM iris_data_packed_summary;

1 rows affected.


source_table,output_table,dependent_varname,independent_varname,dependent_vartype,buffer_size,class_values,num_rows_processed,num_missing_rows_skipped,grouping_cols
iris_data,iris_data_packed,class_text,attributes,character varying,10,"[u'Iris_setosa', u'Iris_versicolor']",52,0,None


# 4. Grouping

Run the preprocessor with grouping by state:

In [11]:
%%sql
DROP TABLE IF EXISTS iris_data_packed, iris_data_packed_summary, iris_data_packed_standardization;

SELECT madlib.minibatch_preprocessor('iris_data',         -- Source table
                                     'iris_data_packed',  -- Output table
                                     'class_text',        -- Dependent variable
                                     'attributes',        -- Independent variables
                                     'state'              -- Grouping
                                     );

SELECT * FROM iris_data_packed ORDER BY state, __id__;

Done.
1 rows affected.
5 rows affected.


__id__,state,dependent_varname,independent_varname
0,Alaska,"[[0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0]]","[[1.26030711687938, -1.615325368523, 1.10943660794792, 1.24354000843452], [1.10129640587123, -0.126074175104234, 1.2524188915498, 1.05700900716934], [0.306242850830503, -0.977074857057813, 0.680489757142278, 0.497416003373807], [0.942285694863087, -1.615325368523, 0.751980898943218, 0.310885002108629], [0.783274983854942, 0.0866759953841608, 0.894963182545097, 1.24354000843452], [-0.806832126226518, 0.299426165872556, -1.03529764608027, -1.36789400927797], [-0.488810704210227, 1.78867735929132, -0.963806504279335, -1.18136300801279], [-1.60188568126725, 0.512176336360951, -1.17827992968215, -1.18136300801279], [-0.965842837234665, 0.0866759953841608, -1.10678878788121, -0.994832006747614], [-0.647821415218373, 1.15042684782613, -1.17827992968215, -0.994832006747614], [-0.647821415218373, -2.04082570949979, 0.394525189938519, 0.310885002108629], [2.05536067192011, 0.299426165872556, 1.03794546614698, 1.05700900716934], [-0.647821415218373, 0.512176336360951, -1.24977107148309, -1.18136300801279]]"
1,Alaska,"[[0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0]]","[[1.41931782788752, 0.724926506849345, 1.2524188915498, 1.4300710096997], [-0.647821415218373, 1.15042684782613, -0.963806504279335, -0.435239002952081], [0.624264272846795, -0.551574516081023, 0.823472040744157, 0.310885002108629], [-1.4428749702591, -1.4025751980346, -1.17827992968215, -0.994832006747614], [0.306242850830503, -0.126074175104234, 0.466016331739459, 0.870478005904162], [1.89634996091196, -0.126074175104234, 1.18092774974886, 0.870478005904162], [-0.32979999320208, -0.551574516081023, 0.680489757142278, 1.05700900716934], [0.46525356183865, -0.338824345592629, 1.10943660794792, 0.870478005904162], [0.306242850830503, 0.0866759953841608, 1.10943660794792, 1.24354000843452], [-0.488810704210227, 0.93767667733774, -1.03529764608027, -1.18136300801279], [-0.488810704210227, 1.78867735929132, -0.749333078876516, -0.808301005482437], [0.147232139822357, 1.15042684782613, -1.17827992968215, -1.18136300801279], [-1.60188568126725, 0.0866759953841608, -1.17827992968215, -1.18136300801279]]"
2,Alaska,"[[0.0, 1.0]]","[[-0.806832126226518, -1.18982502754621, 0.25154290633664, 0.310885002108629]]"
0,Tennessee,"[[0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0]]","[[-0.0286196553591748, -1.22176567731394, 0.412632633639227, 0.22669394242252], [-0.207492501354026, 1.25994585473, -1.12079945083087, -1.19014319771823], [0.507998882625381, -0.839963903153331, 0.621737008794241, 0.580903227457708], [-0.922983885333435, 0.687243193489089, -1.12079945083087, -1.19014319771823], [1.04461742060994, -0.0763603548321211, 1.03994575910427, 0.935112512492896], [2.11785449657905, 0.114540532248182, 1.10964721748927, 1.11221715501049], [0.507998882625381, -0.649063016073029, 0.552035550409236, 0.580903227457708], [-1.99622096130255, -0.267261241912424, -1.19050090921588, -1.19014319771823], [1.40236311259964, -1.41266656439424, 0.90054284233426, 0.758007869975302], [0.32912603663053, 2.59625206429212, -1.12079945083087, -0.835933912683043], [-0.207492501354026, 1.6417476288906, -1.26020236760088, -0.835933912683043], [-2.1750938072974, -0.0763603548321211, -1.39960528437089, -1.36724784023582], [-0.0286196553591748, -1.22176567731394, 0.482334092024232, 0.403798584940115]]"
1,Tennessee,"[[0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0]]","[[0.865744574615085, 0.687243193489089, 0.970244300719264, 1.28932179752808], [-0.0286196553591748, -0.839963903153331, 0.900542842334

Review the output summary table:

In [12]:
%%sql
SELECT * FROM iris_data_packed_summary;

1 rows affected.


source_table,output_table,dependent_varname,independent_varname,dependent_vartype,buffer_size,class_values,num_rows_processed,num_missing_rows_skipped,grouping_cols
iris_data,iris_data_packed,class_text,attributes,character varying,13,"[u'Iris_setosa', u'Iris_versicolor']",52,0,state


Review the output standardization table:

In [13]:
%%sql
SELECT * FROM iris_data_packed_standardization ORDER BY state;

2 rows affected.


state,mean,std
Alaska,"[5.40740740740741, 2.95925925925926, 2.94814814814815, 0.833333333333333]","[0.628888452645665, 0.470034875978888, 1.39877469405147, 0.536103914747325]"
Tennessee,"[5.516, 3.04, 3.108, 0.872]","[0.55905634778617, 0.523832034148353, 1.43469021046357, 0.564637937088893]"


# 5.  Integer dependent variable for classification

If the depedent variable is scalar integer, and you have not already encoded it, you can ask the preprocessor to encode it for you:

In [14]:
%%sql
DROP TABLE IF EXISTS iris_data_packed, iris_data_packed_summary, iris_data_packed_standardization;

SELECT madlib.minibatch_preprocessor('iris_data',         -- Source table
                                     'iris_data_packed',  -- Output table
                                     'class',             -- Integer dependent variable
                                     'attributes',        -- Independent variables
                                     NULL,                -- Grouping
                                     NULL,                -- Buffer size
                                     TRUE                 -- Encode scalar int dependent variable
                                     );

SELECT * FROM iris_data_packed ORDER BY __id__;

Done.
1 rows affected.
2 rows affected.


__id__,dependent_varname,independent_varname
0,"[[0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0]]","[[0.902445979718656, -1.60171977854818, 0.687391438518589, 0.269061417385597], [0.401443941151707, -0.798930106411465, 0.334883008509056, 0.269061417385597], [0.568444620674023, -0.598232688377286, 0.757893124520495, 0.269061417385597], [0.0674425821070736, -1.20032494247982, 0.475886380512869, 0.269061417385597], [-0.934561495026824, -1.20032494247982, 0.193879636505243, 0.269061417385597], [-1.76956489263841, 0.405254401793609, -1.21615408353289, -1.18456909732025], [0.568444620674023, -0.598232688377286, 0.616889752516682, 0.632469046062059], [-0.767560815504508, 1.00734665589615, -1.21615408353289, -1.00286528298202], [-0.0995580974152422, 1.4087414919645, -1.07515071152907, -1.18456909732025], [-0.767560815504508, 1.00734665589615, -1.00464902552717, -0.457753839967327], [-0.600560135982193, 1.60943890999868, -1.00464902552717, -1.18456909732025], [-0.934561495026824, 0.20455698375943, -1.07515071152907, -1.36627291165848], [0.234443261629389, 0.00385956572525086, 1.03989986852812, 1.17758048907675], [-0.0995580974152422, 1.81013632803286, -1.21615408353289, -0.821161468643789], [-1.76956489263841, 0.00385956572525086, -1.21615408353289, -1.18456909732025], [-1.60256421311609, -1.401022360514, -1.21615408353289, -1.00286528298202], [-1.10156217454914, 0.806649237861967, -1.00464902552717, -1.18456909732025], [-0.767560815504508, 0.405254401793609, -1.28665576953479, -1.18456909732025], [-1.76956489263841, -0.196837852308928, -1.14565239753098, -1.18456909732025], [-1.93656557216072, 0.00385956572525086, -1.3571574555367, -1.36627291165848], [1.06944665924097, -0.196837852308928, 1.18090324053193, 0.995876674738521], [0.568444620674023, 2.01083374606704, -1.28665576953479, -1.18456909732025], [0.401443941151707, 2.81362341820376, -1.07515071152907, -0.821161468643789], [0.0674425821070736, -0.999627524445644, 0.687391438518589, 0.81417286040029], [0.902445979718656, -0.196837852308928, 1.03989986852812, 1.17758048907675], [0.568444620674023, -0.798930106411465, 0.687391438518589, 0.632469046062059]]"
1,"[[0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0]]","[[0.902445979718656, 0.806649237861967, 1.03989986852812, 1.35928430341498], [0.902445979718656, -0.598232688377286, 1.46290998453956, 1.35928430341498], [-0.0995580974152422, 0.00385956572525086, 1.03989986852812, 1.17758048907675], [0.234443261629389, 0.00385956572525086, 0.757893124520495, 0.81417286040029], [0.0674425821070736, -1.20032494247982, 0.546388066514775, 0.450765231723828], [1.23644733876329, -1.60171977854818, 1.03989986852812, 1.17758048907675], [-1.10156217454914, 0.00385956572525086, -1.14565239753098, -1.00286528298202], [1.4034480182856, -1.401022360514, 0.969398182526215, 0.81417286040029], [1.4034480182856, 0.605951819827788, 1.18090324053193, 1.35928430341498], [-0.600560135982193, 0.806649237861967, -1.07515071152907, -1.18456909732025], [0.401443941151707, -0.397535270343108, 1.03989986852812, 0.81417286040029], [-0.767560815504508, 0.806649237861967, -1.07515071152907, -1.18456909732025], [1.06944665924097, 0.00385956572525086, 1.11040155453003, 0.995876674738521], [0.234443261629389, -0.999627524445644, 0.616889752516682, 0.450765231723828], [0.0674425821070736, 1.00734665589615, -1.21615408353289, -1.18456909732025], [2.07145073637487, 0.20455698375943, 0.969398182526215, 0.995876674738521], [0.73544530019634, 0.00385956572525086,

Review output summary table:

In [16]:
%%sql
SELECT * FROM iris_data_packed_summary;

1 rows affected.


source_table,output_table,dependent_varname,independent_varname,dependent_vartype,buffer_size,class_values,num_rows_processed,num_missing_rows_skipped,grouping_cols
iris_data,iris_data_packed,class,attributes,integer,26,"[1, 2]",52,0,None
